Trees in Amsterdam
---

Uses the Amsterdam open geodata to find the unique and monumental trees in Amsterdam.

Source:  
https://maps.amsterdam.nl/open_geodata/

### install requirements

In [1]:
%%capture
!python -m pip install --upgrade pip ipyleaflet geopy pandas

### import the third-party packages

In [2]:
# https://github.com/jupyter-widgets/ipywidgets
from ipywidgets import HTML, Layout

# https://github.com/pandas-dev/pandas
import pandas as pd

# https://github.com/geopy/geopy
import geopy
from geopy.distance import geodesic, Distance, Point
from geopy.geocoders import Nominatim

# of https://ipyleaflet.readthedocs.io/en/latest/
from ipyleaflet import Map, Polyline, Marker, DivIcon, AwesomeIcon, LayersControl, LayerGroup, FullScreenControl

### unpack the data

In [3]:
import zipfile
from pathlib import Path

zipped_data = Path('./amsterdamse_bomen_data.zip')

with zipfile.ZipFile(zipped_data, 'r') as z:
    for file in z.filelist:
        z.extractall('./data')

amsterdamse_bomen = Path('./data/amsterdamse_bomen.csv')
monumentale_bomen = Path('./data/monumentale_bomen.csv')

### create a map

In [4]:
# query the location
query = "museumplein"

geolocator = Nominatim(user_agent=query)
location = geolocator.geocode(query)
location_lat_lng = (location.latitude, location.longitude)
print(location_lat_lng)

(52.35867345, 4.882823079370884)


In [5]:
leaflet_map = None
leaflet_map = Map(
    center=location_lat_lng,
    zoom=15,
    layout=Layout(width='100wh', height='95vh')
)

# add layer controls
layer_control = LayersControl(position='topright')
leaflet_map.add_control(layer_control)

# fullscreen button
fullscreen_control = FullScreenControl()
leaflet_map.add_control(fullscreen_control)

# leaflet_map

### create a Geo-fence

In [6]:
radius = 0.5
square = (45, 135, 225, 315)   # bearing from center point
points: list[Point] = [geodesic().destination(point=location.point, bearing=b, distance=Distance(radius)) for b in square]
bearings: list[tuple[float, float]] = [(p.latitude, p.longitude) for p in points]

In [7]:
line_list = []
for coords in zip(bearings, (bearings[1:] + [bearings[0]])): # rotate +1
    # coords: ((lat, lon), (lat, lon))
    line = Polyline(locations=coords, color='blue', fill=False)
    line_list.append(line)
    
line_group = LayerGroup(layers=line_list, name="Geo-fence")
leaflet_map.add_layer(line_group)

# leaflet_map

### create a filter to exclude data outside the Geo-fence

In [8]:
# max lat, lng of the square
longitudes = [p.longitude for p in points]
latitudes = [p.latitude for p in points]
max_lng, min_lng = max(longitudes), min(longitudes)
max_lat, min_lat = max(latitudes), min(latitudes)

In [9]:
# create a filter for items in the square

def in_square(max_lat, min_lat, max_lng, min_lng, df):
    return df[
        (max_lat > df['LAT'])     # smaller than max latitude
        & (df['LAT'] > min_lat)   # larger  than min latidude
        & (max_lng > df['LNG'])   # smaller than max longitude
        & (df['LNG'] > min_lng)   # larger  thanmin longitude
    ]

### read the normal tree data

In [10]:
bomen_df = pd.read_csv(amsterdamse_bomen, delimiter=';')
print(f'rows: {len(bomen_df)}')

filtered = in_square(max_lat, min_lat, max_lng, min_lng, bomen_df)
print(f'items in filtered: {len(filtered)}')

rows: 259431
items in filtered: 1014


In [11]:
# filter single trees
single_trees = filtered.drop_duplicates(subset=['Soortnaam_NL'], keep=False)

### function to create a-tag

In [12]:
# for creating a-tag

from string import whitespace
from urllib.parse import quote

def remove_whitespace(s: str, replacement: str = '') -> str:
    """util to remove all the whitespaces from a string"""
    for c in str(s):
        if c in whitespace:
            s = s.replace(c, str(replacement))
    return s


def create_atag(lat: float, lng: float, zoom: int = 18) -> str:
    """create a-tag with href that links to google maps"""
    p = Point(lat, lng)
    dec_deg = p.format(deg_char='°', min_char="'", sec_char='"')
    dec_deg = remove_whitespace(dec_deg).replace(',', '+')
    href = f"www.google.com/maps/place/{dec_deg}/@{lat:.4f},{lng:.4f},{zoom}z/"
    href = quote(href, safe='+/')
    return f'<a href="https://{href}" target="_blank">{name}</a>'  # a-tag
    

### create markers for the unique trees

In [13]:
# add trees to the map

# https://ipyleaflet.readthedocs.io/en/latest/api_reference/awesome_icon.html
icon = AwesomeIcon(
    name='tree',
    marker_color='green',
    icon_color='darkgreen',
)

markers = []
for _, (name, lat, lng) in single_trees[['Soortnaam_NL', 'LAT', 'LNG']].iterrows():   # zip-like loop
    # create marker
    marker = Marker(location=(lat, lng), 
           draggable=False, 
           title=name,
           icon=icon,
          )
    # add maps-link in popup
    popup = HTML()
    popup.value = create_atag(lat, lng)
    marker.popup = popup
    # add to list
    markers.append(marker)

try:
    leaflet_map.remove_layer(single_tree_group)
except:
    pass
single_tree_group = LayerGroup(layers=markers, name='Unique trees')
leaflet_map.add_layer(single_tree_group)

# leaflet_map

### create markers for the other trees

In [14]:
# add other trees

icon = DivIcon(
    html="""<div style="color: green; font-size: 1.15em">\N{Deciduous Tree}</div>""",
    icon_size=(0, 0),
)

markers = []
for idx, (name, lat, lng) in filtered[['Soortnaam_NL', 'LAT', 'LNG']].iterrows():   # zip-like loop
    # create a-tag
    create_atag(lat, lng)
    # add marker
    marker = Marker(
        location=(lat, lng),
        title=name,
        icon=icon,
    )
    # add maps-link in popup
    popup = HTML()
    popup.value = create_atag(lat, lng)
    marker.popup = popup
    # add to list
    markers.append(marker)


try:  # re
    leaflet_map.remove_layer(all_trees_group)
except:
    pass
all_trees_group = LayerGroup(layers=markers, name='All trees')
leaflet_map.add_layer(all_trees_group)

# leaflet_map

### read the monumental tree data

In [15]:
monumental_df = pd.read_csv(monumentale_bomen, delimiter=';')
print(f'rows: {len(monumental_df)}')

monumental_filtered = in_square(max_lat, min_lat, max_lng, min_lng, monumental_df)
print(f'items in monumental_filtered: {len(monumental_filtered)}')

rows: 2615
items in monumental_filtered: 3


### create markers for the monumental trees

In [16]:
# add monumental trees

icon = AwesomeIcon(
    name='leaf',
    marker_color='red',
    icon_color='green',
)

markers = []
for idx, (name, lat, lng) in monumental_filtered[['Boomsoort', 'LAT', 'LNG']].iterrows():   # zip-like loop
    # create a-tag
    create_atag(lat, lng)
    # add marker
    marker = Marker(
        location=(lat, lng),
        title=name,
        icon=icon,
    )
    # add maps-link in popup
    popup = HTML()
    popup.value = create_atag(lat, lng)
    marker.popup = popup
    # add to list
    markers.append(marker)


try:
    leaflet_map.remove_layer(monumental_trees_group)
except:
    pass
monumental_trees_group = LayerGroup(layers=markers, name='Monumental trees')
leaflet_map.add_layer(monumental_trees_group)

# leaflet_map

### show result

In [17]:
leaflet_map

Map(center=[52.35867345, 4.882823079370884], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_…

In [18]:
leaflet_map.save('leaflet_map.html', title='\N{Deciduous Tree}')